# Test Plugin Integration

> Test the Whisper plugin with the transcription plugin system

In [1]:
import logging
import json
import numpy as np
from pathlib import Path

# Import PluginManager from the generic plugin system
from cjm_plugin_system.core.manager import PluginManager, PluginError
from cjm_transcription_plugin_system.plugin_interface import TranscriptionPlugin
from cjm_transcription_plugin_system.core import AudioData
from cjm_transcription_plugin_whisper.plugin import WhisperLocalPlugin

## Test Direct Plugin Usage

In [2]:
# Create plugin directly
plugin = WhisperLocalPlugin()

# Check basic properties
print(f"Plugin: {plugin.name} v{plugin.version}")
print(f"Available: {plugin.is_available()}")
print(f"Supported formats: {', '.join(plugin.supported_formats)}")

Plugin: whisper_local v1.0.0
Available: True
Supported formats: wav, mp3, flac, m4a, ogg, webm, mp4, avi, mov


In [3]:
# Get configuration class and defaults
from dataclasses import fields
from cjm_plugin_system.utils.validation import SCHEMA_DESC, SCHEMA_ENUM

config_class = plugin.config_class
defaults = plugin.get_config_defaults()

print("Configuration options:")
print(f"- Config class: {config_class.__name__}")
print(f"- Total properties: {len(fields(config_class))}")
print("\nAvailable models:")
model_field = next(f for f in fields(config_class) if f.name == "model")
for model in model_field.metadata.get(SCHEMA_ENUM, []):
    print(f"  - {model}")

Configuration options:
- Config class: WhisperPluginConfig
- Total properties: 23

Available models:
  - tiny
  - tiny.en
  - base
  - base.en
  - small
  - small.en
  - medium
  - medium.en
  - large
  - large-v1
  - large-v2
  - large-v3


## Test with Plugin Manager

In [4]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Create plugin manager
manager = PluginManager(plugin_interface=TranscriptionPlugin)

In [5]:
# Option 1: Load plugin from module directly (for development)
# This works even without the package being installed
import sys
import os

# Add parent directory to path to import the plugin module
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Import and register the plugin
from cjm_transcription_plugin_whisper.plugin import WhisperLocalPlugin

# Create a temporary module file for the plugin manager to load
temp_plugin_file = Path("temp_whisper_plugin.py")
with open(temp_plugin_file, "w") as f:
    f.write("from cjm_transcription_plugin_whisper.plugin import WhisperLocalPlugin\n")

# Load the plugin
success = manager.load_plugin_from_module(
    str(temp_plugin_file),
    config={"model": "tiny", "device": "cpu"}  # Use tiny model for testing
)

print(f"Plugin loaded: {success}")

# Clean up temp file
temp_plugin_file.unlink()

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Initialized Whisper plugin with model 'tiny' on device 'cpu'
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin from module: whisper_local


Plugin loaded: True


In [6]:
# List loaded plugins
print("Loaded plugins:")
for meta in manager.list_plugins():
    print(f"  - {meta.name} v{meta.version} (enabled: {meta.enabled})")

Loaded plugins:
  - whisper_local v1.0.0 (enabled: True)


In [7]:
# Get plugin configuration (returns a dataclass)
current_config = manager.get_plugin_config("whisper_local")
print("Current Whisper configuration:")
config_subset = {
    "model": current_config.model,
    "device": current_config.device,
    "language": current_config.language,
    "task": current_config.task
}
print(json.dumps(config_subset, indent=2))

Current Whisper configuration:
{
  "model": "tiny",
  "device": "cpu",
  "language": null,
  "task": "transcribe"
}


## Test Configuration Management

In [8]:
# Validate different configurations using dataclass validation
from cjm_plugin_system.utils.validation import dict_to_config
from cjm_transcription_plugin_whisper.plugin import WhisperPluginConfig

test_configs = [
    ({"model": "base"}, "Valid: switching to base model"),
    ({"model": "invalid_model"}, "Invalid: bad model name"),
    ({"model": "tiny", "temperature": 0.5}, "Valid: adjusting temperature"),
    ({"model": "tiny", "temperature": 2.0}, "Invalid: temperature out of range"),
]

for config_update, description in test_configs:
    print(f"{description}")
    print(f"  Config: {config_update}")
    try:
        test_config = dict_to_config(WhisperPluginConfig, config_update, validate=True)
        print(f"  Valid: True")
    except ValueError as e:
        print(f"  Valid: False")
        print(f"  Error: {str(e)[:100]}...")
    print()

Valid: switching to base model
  Config: {'model': 'base'}
  Valid: True

Invalid: bad model name
  Config: {'model': 'invalid_model'}
  Valid: False
  Error: model: 'invalid_model' is not one of ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'me...

Valid: adjusting temperature
  Config: {'model': 'tiny', 'temperature': 0.5}
  Valid: True

Invalid: temperature out of range
  Config: {'model': 'tiny', 'temperature': 2.0}
  Valid: False
  Error: temperature: 2.0 is greater than maximum 1.0...



In [9]:
# Update configuration
new_config = {
    "temperature": 0.2,
    "language": "en",
    "word_timestamps": True
}

success = manager.update_plugin_config("whisper_local", new_config, merge=True)
print(f"Configuration updated: {success}")

if success:
    updated_config = manager.get_plugin_config("whisper_local")
    print("\nUpdated configuration:")
    for key in new_config:
        value = getattr(updated_config, key)
        print(f"  {key}: {value}")

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Initialized Whisper plugin with model 'tiny' on device 'cpu'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: whisper_local


Configuration updated: True

Updated configuration:
  temperature: 0.2
  language: en
  word_timestamps: True


## Test Transcription

In [10]:
from nbdev.config import get_config
from pathlib import Path

config = get_config()
project_dir = config.config_path
test_dir = project_dir/"./test_files/"
audio_path = test_dir/"short_test_audio.mp3"
# audio_path = test_dir/"constitution_01_unitedstates_128kb.mp3"
assert audio_path.exists()

In [11]:
print(f"Transcribing: {audio_path}")
result = manager.execute_plugin("whisper_local", audio_path)
print("Transcription result:")
text = '\n    '.join([seg['text'] for seg in result.segments[:5]])
print(f"  Text:\n    {text}")
print(f"  Segments: {len(result.segments) if result.segments else 0}")
print(f"  Metadata: {result.metadata}")

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Loading Whisper model: tiny


Transcribing: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-whisper/test_files/short_test_audio.mp3


cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Local Whisper model loaded successfully
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Transcribing audio with Whisper tiny
100%|███████████████████████████████████████████████████████████| 2799/2799 [00:01<00:00, 1559.60frames/s]
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Transcription completed: 42 words


Transcription result:
  Text:
    November the 10th Wednesday, 9pm. I'm standing in a dark alley. After waiting several hours,
    the time has come. A woman with long dark hair approaches, I have to act, and fast,
    Before she realizes what has happened, I must find out.
  Segments: 3
  Metadata: {'model': 'tiny', 'language': 'en', 'task': 'transcribe', 'device': 'cpu', 'duration': None}


## Test Plugin Lifecycle

In [12]:
# Test disabling and enabling
print("Testing plugin lifecycle:")

# Disable plugin
manager.disable_plugin("whisper_local")
print(f"Plugin disabled: {not manager.plugins['whisper_local'].enabled}")

# Try to execute while disabled (should fail)
try:
    manager.execute_plugin("whisper_local", audio_path)
except PluginError as e:
    print(f"Expected error: {e}")

# Re-enable plugin
manager.enable_plugin("whisper_local")
print(f"Plugin re-enabled: {manager.plugins['whisper_local'].enabled}")

Testing plugin lifecycle:
Plugin disabled: True
Expected error: Cannot execute plugin: disabled
Plugin re-enabled: True


In [13]:
# Clean up
print("\nCleaning up...")
manager.unload_plugin("whisper_local")
print(f"Plugins loaded: {len(manager.list_plugins())}")

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Unloading Whisper model
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Cleanup completed
cjm_plugin_system.core.manager.PluginManager - INFO - Unloaded plugin: whisper_local



Cleaning up...
Plugins loaded: 0


## Test Entry Point Discovery (After Installation)

In [14]:
# This will only work after the package is installed with pip install -e .
# or pip install cjm-transcription-plugin-whisper

print("Testing entry point discovery:")
manager2 = PluginManager(plugin_interface=TranscriptionPlugin)

# Discover plugins via entry points
discovered = manager2.discover_plugins()
print(f"\nDiscovered {len(discovered)} plugin(s) via entry points:")
for plugin_meta in discovered:
    print(f"  - {plugin_meta.name} v{plugin_meta.version} from {plugin_meta.package_name}")

# Load discovered plugins
for plugin_meta in discovered:
    if plugin_meta.name == "whisper_local":
        success = manager2.load_plugin(plugin_meta, config={"model": "tiny"})
        print(f"\nLoaded {plugin_meta.name}: {success}")

cjm_plugin_system.core.manager.PluginManager - INFO - Discovered plugin: whisper_local v0.0.5 from package cjm-transcription-plugin-whisper
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Initialized Whisper plugin with model 'tiny' on device 'cuda'
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin: whisper_local


Testing entry point discovery:

Discovered 1 plugin(s) via entry points:
  - whisper_local v0.0.5 from cjm-transcription-plugin-whisper

Loaded whisper_local: True


In [15]:
manager2.update_plugin_config('whisper_local', config={"model": "large-v3", "device": "cuda"})
manager2.get_plugin_config('whisper_local')

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Initialized Whisper plugin with model 'large-v3' on device 'cuda'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: whisper_local


WhisperPluginConfig(model='large-v3', device='cuda', language=None, task='transcribe', temperature=0.0, temperature_increment_on_fallback=0.2, beam_size=5, best_of=5, patience=1.0, length_penalty=None, suppress_tokens='-1', initial_prompt=None, condition_on_previous_text=False, fp16=True, compression_ratio_threshold=2.4, logprob_threshold=-1.0, no_speech_threshold=0.6, word_timestamps=False, prepend_punctuations='"\'“¿([{-', append_punctuations='"\'.。,，!！?？:：”)]}、', threads=0, model_dir=None, compile_model=False)

In [16]:
audio_path = test_dir/"short_test_audio.mp3"
# audio_path = test_dir/"constitution_01_unitedstates_128kb.mp3"
assert audio_path.exists()

In [17]:
print(f"Transcribing: {audio_path}")
result = manager2.execute_plugin("whisper_local", audio_path)
print("Transcription result:")
text = '\n    '.join([seg['text'] for seg in result.segments[:5]])
print(f"  Text:\n    {text}")
print(f"  Segments: {len(result.segments) if result.segments else 0}")
print(f"  Metadata: {result.metadata}")

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Loading Whisper model: large-v3


Transcribing: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-whisper/test_files/short_test_audio.mp3


cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Local Whisper model loaded successfully
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Transcribing audio with Whisper large-v3


Detected language: English


100%|███████████████████████████████████████████████████████████| 2799/2799 [00:01<00:00, 2626.07frames/s]
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Transcription completed: 44 words


Transcription result:
  Text:
    november tenth wednesday nine p m i am standing in a dark alley after waiting several hours the time has come
    a woman with long dark hair approaches i have to act and fast before she realizes what has happened i must find out
  Segments: 2
  Metadata: {'model': 'large-v3', 'language': 'en', 'task': 'transcribe', 'device': 'cuda', 'duration': None}


In [18]:
manager2.get_plugin('whisper_local').cleanup()

cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Unloading Whisper model
cjm_transcription_plugin_whisper.plugin.WhisperLocalPlugin - INFO - Cleanup completed
